# Exploratory Analysis

* $RQ_1$: How does temporal proximity affect sharing behaviour during protest demonstrations?
* $RQ_2$: How do source attributes like the poster's gender affect sharing behaviour during protest demonstrations?
* $RQ_3$: Is there evidence of a "backfire effect" where misleading posts that *don't* have a credibility indicator are assumed to be true and more likely to be reshared.


In [16]:
library(lme4)
library(AER)
library(dplyr)
library(purrr)
library(magrittr)
library(ggplot2)
require(ggiraph)
require(plyr)
require(moonBook)   # for use of data radial
require(GGally)
require(reshape2)
require(compiler)
require(parallel)
require(boot)
require(lattice)

data <- read.csv("../../data/processed/607d6b3929befce813fe5ba2-rumour-results.csv")
data$reshared <- as.integer(as.logical(data$reshared))
head(data)

Loading required package: moonBook

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘moonBook’”


,user_id,condition,timeSubmitted,secondsTaken,id,rumour,code,evidence,warning,reshared,clickedWarning,timestamp,mockGender
,<dbl>,<chr>,<chr>,<int>,<dbl>,<chr>,<chr>,<chr>,<chr>,<int>,<chr>,<int>,<chr>
1,7.057603e+18,treatment,2021-05-24 10:57:41.980000+00:00,120,1.283487e+18,R1,denies,high,True,1,False,2,female
2,7.057603e+18,treatment,2021-05-24 10:57:41.980000+00:00,120,1.284272e+18,R1,neutral,high,False,0,False,2,female
3,7.057603e+18,treatment,2021-05-24 10:57:41.980000+00:00,120,1.283527e+18,R1,questions,high,False,1,False,4,female
4,7.057603e+18,treatment,2021-05-24 10:57:41.980000+00:00,120,1.283565e+18,R1,denies,high,True,1,True,5,female
5,7.057603e+18,treatment,2021-05-24 10:57:41.980000+00:00,120,1.283799e+18,R1,questions,high,False,0,False,6,male
6,7.057603e+18,treatment,2021-05-24 10:57:41.980000+00:00,120,1.283523e+18,R1,affirms,high,False,1,False,7,male


### Logistic regression

Alt add an interaction between warning and evidence (if we expect there to be different effects for putting a cred indicator on a post denying a high evidence rumour vs affirming a low evidence rumour.

In [28]:
md <- glmer(reshared ~ mockGender + timestamp + warning + code * evidence + (1 | user_id) + (1 | id),
            data = data,
            family = binomial, 
            control=glmerControl(optimizer="bobyqa",optCtrl=list(maxfun=2e5))
           )
summary(md)

Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: reshared ~ mockGender + timestamp + warning + code * evidence +  
    (1 | user_id) + (1 | id)
   Data: data
Control: glmerControl(optimizer = "bobyqa", optCtrl = list(maxfun = 2e+05))

     AIC      BIC   logLik deviance df.resid 
   323.3    376.6   -148.6    297.3      435 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-1.2526 -0.3520 -0.2235 -0.1111  8.2647 

Random effects:
 Groups  Name        Variance Std.Dev.
 id      (Intercept) 0.4643   0.6814  
 user_id (Intercept) 0.8819   0.9391  
Number of obs: 448, groups:  id, 153; user_id, 8

Fixed effects:
                          Estimate Std. Error z value Pr(>|z|)    
(Intercept)               -0.66269    0.58463  -1.134  0.25700    
mockGendermale            -0.16318    0.33972  -0.480  0.63099    
timestamp                 -0.04493    0.01123  -4.002 6.27e-05 ***
warningTrue          